**Satellite Simulation** \
(Divya Ramachandran,Hadis  Banafsheh, Suvan Kumar)

In [1]:
#SNODAS Dataset

In [ ]:
import os
import requests
import zipfile
import geopandas as gpd
from shapely.geometry import Polygon

from datetime import datetime, timedelta, timezone

start = datetime(2024, 1, 1, tzinfo=timezone.utc)
end = datetime(2024, 5, 30, tzinfo=timezone.utc)
frame_duration = timedelta(days=1)
num_frames = int(1 + (end - start) / frame_duration)

# automatically download/extract watershed boundary shapefiles
for wbd in [10]:
    if not os.path.isfile(f"WBD_{wbd}_HU2_Shape/Shape/WBDHU2.shp"):
        print(f"Downloading/extracting WBD {wbd}")
        r = requests.get(
            f"https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/WBD/HU2/Shape/WBD_{wbd}_HU2_Shape.zip"
        )
        with open(f"WBD_{wbd}_HU2_Shape.zip", "wb") as zip_file:
            zip_file.write(r.content)
        with zipfile.ZipFile(f"WBD_{wbd}_HU2_Shape.zip", 'r') as zip_file:
            zip_file.extractall(f"WBD_{wbd}_HU2_Shape")

mo_basin = gpd.read_file("WBD_10_HU2_Shape/Shape/WBDHU2.shp")
mo_basin = gpd.GeoSeries(Polygon(mo_basin.iloc[0].geometry.exterior), crs="EPSG:4326")

# access us states shapefile
us_map = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_20m.zip")
conus = us_map[~us_map.STUSPS.isin(["AK", "HI", "PR"])].to_crs("EPSG:4326")
# note: requires gdal (has platform-specific binaries)
# recommend conda distribution: `conda install -c conda-forge gdal`
from osgeo import gdal

from datetime import datetime
import gzip
import os
import requests
import shutil
import tarfile
import tempfile

import pandas as pd
import rioxarray as rxr
import xarray as xr

dates = pd.date_range(start, end)
snodas_dir = "SNODAS"
if not os.path.exists(snodas_dir):
    os.makedirs(snodas_dir)

for date in dates:
    # Make sure the date is timezone-aware
    date = datetime(date.year, date.month, date.day, tzinfo=timezone.utc)

    file_label = f"us_ssmv11034tS__T0001TTNATS{date.strftime('%Y')}{date.strftime('%m')}{date.strftime('%d')}05HP001.nc"
    
    # check if file already exists
    if os.path.isfile(os.path.join(snodas_dir, file_label)):
        print("Skipping " + file_label)
        continue
    print("Processing " + file_label)
    dir_label = f"SNODAS_{date.strftime('%Y%m%d')}"
    # request the .tar file
    r = requests.get(
        "https://noaadata.apps.nsidc.org/NOAA/G02158/masked/" + 
        f"{date.strftime('%Y')}/{date.strftime('%m')}_{date.strftime('%b')}/" +
        dir_label + ".tar"
    )
    # do work in temp directory
    with tempfile.TemporaryDirectory() as tmp_dir:
        # save the .tar file
        with open(os.path.join(tmp_dir, dir_label + ".tar"), "wb") as tar_file:
            tar_file.write(r.content)
        # open and extract the .tar file
        with tarfile.open(os.path.join(tmp_dir, dir_label + ".tar"), "r") as tar_file:
            tar_file.extractall(tmp_dir)
        # iterate through all files
        for filename in os.listdir(tmp_dir):
            if os.path.isfile(os.path.join(tmp_dir, filename)) and all(s in filename for s in ["v11034", ".dat.gz"]):
                # unzip the swe .gz file
                with gzip.open(os.path.join(tmp_dir, filename), "rb") as gz_in:
                    with open(os.path.join(tmp_dir, filename.replace(".gz", "")), "wb") as gz_out:
                        shutil.copyfileobj(gz_in, gz_out)
                # write the .hdr file
                with open(os.path.join(tmp_dir, filename.replace(".dat.gz", ".hdr")), "w") as hdr_file:
                    hdr_file.write(
                        "ENVI\n"
                        "samples = 6935\n" +
                        "lines = 3351\n" +
                        "bands = 1\n" +
                        "header offset = 0\n" + 
                        "file type = ENVI Standard\n" + 
                        "data type = 2\n" +
                        "interleave = bsq\n" +
                        "byte order = 1"
                    )
                # run the gdal translator
                if date < datetime(2013, 10, 1, tzinfo=timezone.utc):
                    command = " ".join([
                        "gdal_translate",
                        "-of NetCDF",
                        "-a_srs EPSG:4326",
                        "-a_nodata -9999",
                        "-a_ullr -124.73375000000000 52.87458333333333 -66.94208333333333 24.94958333333333",
                        os.path.join(tmp_dir, filename.replace(".dat.gz", ".dat")),
                        os.path.join(snodas_dir, filename.replace(".dat.gz", ".nc"))
                    ])
                else:
                    command = " ".join([
                        "gdal_translate",
                        "-of NetCDF",
                        "-a_srs EPSG:4326",
                        "-a_nodata -9999",
                        "-a_ullr -124.73333333333333 52.87500000000000 -66.94166666666667 24.95000000000000",
                        os.path.join(tmp_dir, filename.replace(".dat.gz", ".dat")),
                        os.path.join(snodas_dir, filename.replace(".dat.gz", ".nc"))
                    ])
                if os.system(command) > 0: 
                    print(f"Error processing command `{command}`")

print("Merging datasets...")
ds = xr.combine_by_coords(
    [
        rxr.open_rasterio(
            os.path.join(
                snodas_dir, 
                f"us_ssmv11034tS__T0001TTNATS{date.strftime('%Y')}{date.strftime('%m')}{date.strftime('%d')}05HP001.nc"
            )
        ).rio.clip(mo_basin.envelope).assign_coords(time=date).expand_dims(dim="time")
        for date in dates
    ], 
    combine_attrs="drop_conflicts"
).sel(band=1).rename_vars({"Band1": "SWE"})
ds.SWE.attrs["long_name"] = "Snow Water Equivalent"
ds.SWE.attrs["units"] = "mm"

print("Writing snodas-merged.nc")
ds.to_netcdf(os.path.join(snodas_dir, "snodas-merged.nc"))


Downloading/extracting WBD 10
Processing us_ssmv11034tS__T0001TTNATS2024010105HP001.nc
Input file size is 6935, 3351
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing us_ssmv11034tS__T0001TTNATS2024010205HP001.nc
Input file size is 6935, 3351
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing us_ssmv11034tS__T0001TTNATS2024010305HP001.nc
Input file size is 6935, 3351
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing us_ssmv11034tS__T0001TTNATS2024010405HP001.nc
Input file size is 6935, 3351
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing us_ssmv11034tS__T0001TTNATS2024010505HP001.nc
Input file size is 6935, 3351
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing us_ssmv11034tS__T0001TTNATS2024010605HP001.nc
Input file size is 6935, 3351
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing us_ssmv11034tS__T0001TTNATS2024010705HP001.nc
Input file size is 6935, 

In [ ]:
#Down sampling the resoultio from 1km to 100km
from rasterio.enums import Resampling
swe = ds.SWE.rio.reproject(ds.SWE.rio.crs, shape=(ds.SWE.shape[1]//100, ds.SWE.shape[2]//100), resampling=Resampling.average)

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime

# Ensure dimensions of size 1 are squeezed out
if 'band' in swe.dims and swe.sizes['band'] == 1:
    swe = swe.squeeze('band')

# Mask NaN values in SWE
swe_masked = swe.where(~np.isnan(swe))

# Calculate the absolute SWE difference between consecutive time steps
swe_diff_abs = abs(swe_masked.diff(dim='time')).where(~np.isnan(swe_masked.diff(dim='time')))

# Add a zero difference for the first time step to match the length
swe_diff_abs = xr.concat([xr.zeros_like(swe.isel(time=0)), swe_diff_abs], dim='time')

# Set the threshold for SWE change
threshold = 15

# Create a list to store the results
results = []

# Iterate through the dataset and check for each grid cell
for t in range(len(swe_diff_abs.time)):
    # Extract the SWE difference for the current time step
    swe_diff_time = swe_diff_abs.isel(time=t)
    
    # Check if any grid cell exceeds the threshold
    grid_cells_exceeding_threshold = swe_diff_time.where(swe_diff_time > threshold, drop=True)
    
    if not grid_cells_exceeding_threshold.isnull().all():
        # Get the date of the current time step
        date = swe['time'].isel(time=t).values.astype('datetime64[D]')
        
        # Convert to a Python datetime object to access 'year'
        date_obj = pd.to_datetime(date)
        
        # Calculate the number of days since the start of the year (January 1st)
        day_from_start = (date_obj - datetime(date_obj.year, 1, 1)).days + 1
        
        # Append the results
        results.append([date, day_from_start])

# Convert results to a DataFrame
dates_df = pd.DataFrame(results, columns=["Date", "Days_from_start"])

# Calculate the interarrival days by taking the difference between consecutive dates
dates_df['Interarrival_days'] = dates_df['Date'].diff().fillna(pd.Timedelta(0)).dt.days

# Remove the first row
dates_df = dates_df.iloc[1:].reset_index(drop=True)

# Save the DataFrame to a CSV file
# output_csv = 'dates_with_interarrival.csv'
# dates_df.to_csv(output_csv, index=False)

# Print confirmation and show a preview of the DataFrame
# print(f"Dates where SWE change exceeds the threshold saved to '{output_csv}'.")
print(dates_df.head())


In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta

def bootstrap_interarrivals(dates_df, n_iterations=1000):
    """
    Perform bootstrap sampling on the inter-arrival times and calculate the mean and confidence intervals.

    Parameters:
    dates_df (DataFrame): DataFrame containing the inter-arrival days data.
    n_iterations (int): The number of bootstrap samples to generate.

    Returns:
    tuple: A tuple containing the lower and upper bounds of the 95% confidence interval for the mean.
    """
    # Create a list to store the bootstrap samples
    bootstrap_samples = []
    bootstrap_means = []

    # Perform bootstrap sampling
    for i in range(n_iterations):
        # Sample with replacement from the 'Interarrival_days' column
        sample = dates_df['Interarrival_days'].sample(frac=1, replace=True)
        bootstrap_samples.append(sample)  
        sample_mean = np.mean(sample) 
        bootstrap_means.append(sample_mean)

    # Calculate the 95% confidence interval (using the 2.5th and 97.5th percentiles)
    lower_bound = np.percentile(bootstrap_means, 2.5)
    upper_bound = np.percentile(bootstrap_means, 97.5)

    return lower_bound, upper_bound


def image_request(dates_df):
    """
    Function to get a random inter-arrival time from bootstrap samples.
    This function randomly selects a sample from the 'Interarrival_days' column in the bootstrap dataset.
    
    Parameters:
    dates_df (DataFrame): DataFrame containing the inter-arrival days data.
    
    Returns:
    timedelta: The inter-arrival time (in days) for the next satellite request.
    """
    # Sample a random inter-arrival time from the bootstrap samples
    inter_arrival_time = np.random.choice(dates_df['Interarrival_days'])
    
    # Convert to plain Python int to avoid the TypeError with numpy.int64
    inter_arrival_time = int(inter_arrival_time)
    
    return timedelta(days=inter_arrival_time)

# Example usage:
# Assuming you have your `dates_df` already loaded
# Perform bootstrap analysis
lower_bound, upper_bound = bootstrap_interarrivals(dates_df, n_iterations=1000)
print(f"95% Confidence Interval for the Mean: ({lower_bound}, {upper_bound})")

# Now, call image_request function with dates_df
image_delta = image_request(dates_df)
print(f"Next Image Request will be in: {image_delta}")


Changes made on 7th December

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tatc
import rioxarray as rxr
#import tatc[examples]
from tatc.schemas import TwoLineElements
from tatc.schemas import Point
from tatc.analysis import collect_observations
from tatc import utils
from tatc.schemas import Instrument
from tatc.schemas import WalkerConstellation, SunSynchronousOrbit
from tatc.utils import (
    swath_width_to_field_of_regard,
    along_track_distance_to_access_time,
)
import datetime
from tatc.analysis import collect_multi_observations
from tatc.schemas import Satellite
from datetime import datetime, timedelta, timezone


Defining Satellites and TLEs

In [97]:
noaa20_tle = [
    "1 43013U 17073A   22195.78278435  .00000038  00000+0  38919-4 0  9996",
    "2 43013  98.7169 133.9110 0001202  63.8768 296.2532 14.19561306241107",
]
CAPELLA_13_tle= [ 
    "1 60419U 24142A   24326.56272859  .00014736  00000+0  16272-2 0  9994",
    "2 60419  53.0086  37.0038 0001423  85.1678 274.9464 14.87633793 15171",
]

CAPELLA_14_tle= [ 
    "1 59444U 24066C   24326.41256276  .00017846  00000+0  17602-2 0  9997",
    "2 59444  45.6101 354.1920 0001417  63.9982 296.1042 14.91842553 26306",
]

CAPELLA_15_tle= [ 
    "1 60544U 24149CE  24326.63431561  .00009918  00000+0  86496-3 0  9997",
    "2 60544  97.7350  40.7817 0005870 174.5984 185.5302 14.97041464 14455",
]

CAPELLA_11_tle= [ 
    "1 57693U 23126A   24326.55257719  .00001335  00000+0  20605-3 0  9999",
    "2 57693  53.0084 132.6964 0004021 153.2687 206.8497 14.79537557 67352",
]

CAPELLA_09_tle= [ 
    "1 55910U 23035C   24326.19469929  .00080387  00000+0  28081-2 0  9995",
    "2 55910  43.9854 265.6943 0016492 143.7069 216.4921 15.28530799 92535",
]

CAPELLA_10_tle= [ 
    "1 55909U 23035B   24326.33838605  .00062564  00000+0  26624-2 0  9998",
    "2 55909  43.9916 278.3602 0014472 119.8274 240.4032 15.22108149 92391",
]

# Design Alternatives

GRACE_FO_1_tle = [             
    "1 43476U 18047A   24338.59478596  .00008469  00000+0  27203-3 0  9990",
    "2 43476  88.9749 275.5963 0014875 124.0249 236.2418 15.32050354363786",
]

GRACE_FO_2_tle = [             
    "1 43477U 18047B   24338.52972907  .00008473  00000+0  27213-3 0  9995",
    "2 43477  88.9748 275.6044 0014834 124.1677 236.0984 15.32053872363771",
]

capella_09_orbit = TwoLineElements(tle=CAPELLA_09_tle)
capella_10_orbit = TwoLineElements(tle=CAPELLA_10_tle)
capella_11_orbit = TwoLineElements(tle=CAPELLA_11_tle)
capella_13_orbit = TwoLineElements(tle=CAPELLA_13_tle)
capella_14_orbit = TwoLineElements(tle=CAPELLA_14_tle)
capella_15_orbit = TwoLineElements(tle=CAPELLA_15_tle)
grace_fo_1_orbit = TwoLineElements(tle=GRACE_FO_1_tle)
grace_fo_2_orbit = TwoLineElements(tle=GRACE_FO_2_tle)

#noaa20_orbit = TwoLineElements(tle=noaa20_tle)

# Defining Instrument
capella_sar_for = utils.swath_width_to_field_of_regard(500e3, 30e3)
capella_sar = Instrument(name="SAR", field_of_regard=capella_sar_for)

# Instrument for GRACE
grace_LRR_for = utils.swath_width_to_field_of_regard(490e3, 300e3)
grace_LRR = Instrument(name="LRR", field_of_regard=grace_LRR_for)

# Defining Satellite

Capella_09 = Satellite(name="Capella 9", orbit=capella_09_orbit, instruments=[capella_sar])
Capella_10 = Satellite(name="Capella 10", orbit=capella_10_orbit, instruments=[capella_sar])
Capella_11 = Satellite(name="Capella 11", orbit=capella_11_orbit, instruments=[capella_sar])
Capella_13 = Satellite(name="Capella 13 ", orbit=capella_13_orbit, instruments=[capella_sar])
Capella_14 = Satellite(name="Capella 14", orbit=capella_14_orbit, instruments=[capella_sar])
Capella_15 = Satellite(name="Capella 15", orbit=capella_15_orbit, instruments=[capella_sar])


# Design Alternatives
Grace_fo_1 = Satellite(name="GRACE FO-1 ", orbit=grace_fo_1_orbit, instruments=[grace_LRR])
Grace_fo_2 = Satellite(name="GRACE FO-2 ", orbit=grace_fo_2_orbit, instruments=[grace_LRR])

# Base Constellation and Design alternatives
Capella_const = [Capella_09,Capella_10,Capella_11,Capella_13,Capella_14,Capella_15]
Design_1 = [Capella_09,Capella_10,Capella_11,Capella_13,Capella_14,Capella_15,Grace_fo_1]
Design_2 = [Capella_09,Capella_10,Capella_11,Capella_13,Capella_14,Capella_15,Grace_fo_2]

Obtaining centroid of missouri basin

In [98]:
import os
import requests
import zipfile
import geopandas as gpd
from shapely.geometry import Polygon

path_shp = 'C:\\Users\\dramach6\\ASU Dropbox\\Divya Ramachandran\\SOS Planning\\Downloaded_files\\Mo_basin_shp\\'
# Automatically download/extract watershed boundary shapefiles
for wbd in [10]:
    if not os.path.isfile(path_shp+f"WBD_{wbd}_HU2_Shape/Shape/WBDHU2.shp"):
        print(f"Downloading/extracting WBD {wbd}")
        r = requests.get(
            f"https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/WBD/HU2/Shape/WBD_{wbd}_HU2_Shape.zip"
        )
        with open(path_shp + f"WBD_{wbd}_HU2_Shape.zip", "wb") as zip_file:
            zip_file.write(r.content)
        with zipfile.ZipFile(path_shp+f"WBD_{wbd}_HU2_Shape.zip", 'r') as zip_file:
            zip_file.extractall(path_shp+f"WBD_{wbd}_HU2_Shape")

mo_basin = gpd.read_file(path_shp+"WBD_10_HU2_Shape/Shape/WBDHU2.shp")
mo_basin = gpd.GeoSeries(Polygon(mo_basin.iloc[0].geometry.exterior), crs="EPSG:4326")
mo_basin = mo_basin.to_crs("EPSG:4326")

# Finding centroid of Missouri Basin

centroid = mo_basin.to_crs('+proj=cea').centroid.to_crs(mo_basin.crs)
#centroid = mo_basin.geometry.centroid
centroid

0    POINT (-102.69201 43.50661)
dtype: geometry

Scheduling Arrivals, that is, imaging requests\
Using TAT-C to generate observations

In [ ]:
import numpy as np
from datetime import datetime, timedelta, timezone
import warnings

# def imaging_request():
#     return timedelta(np.random.exponential(1/0.16))

def constellation_capacity():
    return np.random.rand()

# Generating observations

from tatc.schemas import Point
missouri = Point(id=0,latitude=43.50661,longitude=-102.69201)

# Collecting multi observations ( 5 days from start)

def observations(start,counter,constellation):

    #start = datetime(year=2024, month=1, day=1, hour=12, tzinfo=timezone.utc)   
    end = start + timedelta(days=10)
    results = collect_multi_observations(missouri, constellation, start, end) 
    #display(results)  
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)  # Ignore UserWarnings
        next_obs = (results['epoch'][counter]).to_pydatetime()  
    #next_obs = (results['epoch'][counter]).to_pydatetime()
    sat = results['satellite'][counter]
    return (next_obs,sat)

HADIS CODE\
Note 
1. Refer to imaging_requests module above
2. Ensure all arrivals are generated at time 0:0000
3. Change N value for testing, currently it is set to 50, that will take time

In [ ]:
# Input modelling here
# Hadis's code

In [100]:
observations(datetime.now(timezone.utc),0,Capella_const)

(datetime.datetime(2024, 12, 8, 2, 21, 43, 128747, tzinfo=datetime.timezone.utc),
 'Capella 13 ')

Discrete Event Simulation Code

In [ ]:
# Simulation Model

import numpy as np
from datetime import datetime, timedelta, timezone

def simu(constellation):
    t = datetime.now(timezone.utc)
    end = t + timedelta(61)
    image_delta = image_request()
    User_request = t + image_delta
    inf_time = datetime.max.replace(tzinfo=timezone.utc)
    Next_Observation = inf_time
    Next_Opportunity = inf_time
    N = 0
    const_capacity = 0
    threshold = 0.75
    Type = ""
    counter = 0
    window_close = 2
    arr_user_request = []
    arr_obs_completion = []
    i = 0
    expiration_date = []
    request_status = ''
    arr_request_status = []

    k = 20

    print("{:^10}{:^15}{:^35}{:^35}{:^35}{:^25}{:^35}{:^35}{:^10}{:^20}".format("iteration","Type" ,"Time", "delta","Next user Request" ,"Number of active request","Next Opportunity", "Next Observation","Status","Constellation Capacity"))
    print("\n")
    #for i in range(k):
    while t <= end:
        formatted_image_delta = f"{image_delta.days} days, {image_delta.seconds // 3600:02}:{(image_delta.seconds % 3600) // 60:02}:{image_delta.seconds % 60:02}"
        text = "{:^10}{:^15}{:^35}{:^35}{:^35}{:^25}{:^35}{:^35}{:^10}{:^20.2f}"
        print(text.format(i,Type,t.strftime("%Y-%m-%d %H:%M:%S"),formatted_image_delta,User_request.strftime("%Y-%m-%d %H:%M:%S"),N,Next_Opportunity.strftime("%Y-%m-%d %H:%M:%S"),Next_Observation.strftime("%Y-%m-%d %H:%M:%S"),request_status,const_capacity))
        
        t = min(User_request,Next_Observation)
        if t < Next_Observation:
            Type = 'User_Request'
            expiration = t+timedelta(window_close)
            arr_user_request.append(t)
            expiration_date.append(expiration)
            image_delta = image_request()
            User_request = t + image_delta        
            const_capacity = 0
            request_status = 'Active'
            N+=1
            if N>0 and Next_Observation==inf_time:
                counter = 0
                obs = observations(t,counter,constellation)
                #print(obs)
                #Next_Observation = t + obs[0]
                Next_Observation = obs[0]
                Next_Observation_satellite = obs[1]
                #Next_Opportunity = t + obs[0]
                Next_Opportunity = obs[0]
                if Next_Observation > expiration:
                    N = 0
                    Next_Observation = inf_time
                    request_status = 'Rejected'
            arr_request_status.append(request_status)          


        elif t <= expiration:
            Type = 'Obs Opportunity'
            const_capacity = constellation_capacity()
            if const_capacity < 0.75:
                N-=1
                request_status = 'Completed'
                arr_obs_completion.append([t,Next_Observation_satellite,request_status])
                Next_Observation = inf_time
                Next_Opportunity = inf_time
                N = 0
                counter =0
            else:
                counter = 0
                obs = observations(t,counter,constellation)                
                #Next_Observation = t + obs[0]
                Next_Observation = obs[0]
                Next_Observation_satellite = obs[1]
                #Next_Opportunity = t + obs[0]
                Next_Opportunity = obs[0]
                if Next_Observation > expiration:
                    N = 0
                    Next_Observation = inf_time
                    request_status = 'Rejected'

        else:
            request_status = 'Rejected'
            Next_Observation = inf_time
            N = 0


        i = i + 1
      
    return arr_user_request, arr_obs_completion,expiration_date


Creating Dataframe for Output

In [ ]:
import pandas as pd

# Create a DataFrame to store user requests, completion times, and satellite assignments
def create_observation_dataframe(user_requests, obs_completion_data,request_expiration):
    # Initialize lists for the DataFrame
    completion_dates = []
    satellites = []
    status = []    
    
    for req in user_requests:
        # Find the first observation completion that is on or after the user request
        matched_completion = None
        matched_satellite = None
        req_status = None
        for comp, sat, stat in obs_completion_data:
            if req <= comp:
                matched_completion = comp
                matched_satellite = sat
                req_status = stat
                break
        
        # Append the matched values
        completion_dates.append(matched_completion)
        satellites.append(matched_satellite)
        status.append(req_status)
    
    # Create the DataFrame
    df = pd.DataFrame({
        "User Request": user_requests,
        "Request Expiration":request_expiration,
        "Request Status": status,
        "Observation Completion": completion_dates,
        "Satellite": satellites
    })

    # Convert the columns to datetime if they are not already
    df["User Request"] = pd.to_datetime(df["User Request"])
    df["Observation Completion"] = pd.to_datetime(df["Observation Completion"])
    df = df.iloc[:-1]
    
    # Calculate the difference between User Request and Observation Completion  

    df["Time Difference"] = df.apply(
    lambda row: ((row["Observation Completion"] - row["User Request"]).seconds)/3600 
    if pd.notna(row["Observation Completion"]) else np.nan,
    axis=1
    )
    df["Within 2 Days"] = df["Time Difference"].apply(
    lambda x: 1 if pd.notna(x) and x <= 48 else 0
    )    
    
    if len(df) > 0:
        service = df["Within 2 Days"].sum() / len(df)
    else:
        service = np.nan

    # Ensure non-null values exist for Average_TAT calculation
    non_null_time_diff = df["Time Difference"].dropna()
    if len(non_null_time_diff) > 0:
        tat = non_null_time_diff.sum() / len(non_null_time_diff)
    else:
        tat = np.nan

    # Replace NaN values for missing completion dates with a placeholder (e.g., "NA")
    df["Observation Completion"] = df["Observation Completion"].fillna("NA") 
    display(df)   

    return service,tat

Running model for N iterations

In [ ]:
s = []
t = []
mean_tat = []
std_tat = []
mean_service = []
std_service = []
Design = [Capella_const,Design_1,Design_2]
Design_name = ['Capella_const','Grace_fo_1','Grace_fo_2']
N = 50

for j in Design:
   print(j)
   for i in range(N):
      sim = simu(j)
      arr_user_request = sim[0]   
      arr_obs_completion = sim[1]
      # print(len(arr_user_request))
      # print(len(arr_obs_completion))
      expiration_date = sim[2]  
      #arr_user_request = arr_user_request[:-1] 
      df = create_observation_dataframe(arr_user_request, arr_obs_completion,expiration_date)
      s.append(df[0])
      t.append(df[1])

   service = np.array(s).flatten()
   tat = np.array(t).flatten()
   mean_tat.append(np.nanmean(tat))
   std_tat.append(np.nanstd(tat))
   mean_service.append(np.nanmean(service))
   std_service.append(np.nanstd(service))    


Mean and Standard deviation

In [117]:
i = 1
for i in range(len(Design_name)):
    print("\n ")
    print(Design_name[i])
    print(f"Mean TAT(hours): {mean_tat[i]}")
    print(f"Standard Deviation TAT(hours): {std_tat[i]}")
    print(f"Mean Service: {mean_service[i]}")
    print(f"Standard Deviation Service: {std_service[i]}")


 
Capella_const
Mean TAT(hours): 11.702513691794318
Standard Deviation TAT(hours): 2.4541509969616397
Mean Service: 0.9299531775414128
Standard Deviation Service: 0.10633928815717024

 
Grace_fo_1
Mean TAT(hours): 11.252170959727
Standard Deviation TAT(hours): 2.53638786060494
Mean Service: 0.9356670649611826
Standard Deviation Service: 0.10055279454039885

 
Grace_fo_2
Mean TAT(hours): 10.752784908990188
Standard Deviation TAT(hours): 2.4943089685080784
Mean Service: 0.9415207080501201
Standard Deviation Service: 0.0942568122454


Absolute performance \
Prediction and Confidence Interval, 95% confidence levels, N = 50, t = 2.008

In [118]:
for i in range(len(Design_name)):
    print("\n ")
    print(Design_name[i])
    t = 2.008
    confidence_tat = (t * std_tat[i])/np.sqrt(50)
    prediction_tat = (t * std_tat[i])*np.sqrt(1 + (1/50))
    confidence_service = (t * std_service[i])/np.sqrt(50)
    prediction_service = (t * std_service[i])*np.sqrt(1 + (1/50))
    
    Error_threshold_TAT = 0.5 # 30 minute threshold for TAT
    Error_threshold_Service = 0.05 # 0.05 threshold for service levels
    N_tat = (1.96 * std_tat[i]/Error_threshold_TAT)**2
    N_service = (1.96 * std_service[i]/Error_threshold_Service)**2 

    print(f"Confidence Interval TAT(hours): {mean_tat[i] - confidence_tat,mean_tat[i] + confidence_tat }")
    print(f"Prediction Interval TAT(hours): {mean_tat[i] - prediction_tat,mean_tat[i] + prediction_tat}")
    print(f"Confidence Interval Service: {mean_service[i] - confidence_service,mean_service[i] + confidence_service }")
    print(f"Prediction Interval Service: {mean_service[i] - prediction_service,mean_service[i] + prediction_service}") 
    print(f"Number of Samples TAT:{N_tat}") 
    print(f"Number of Samples Service:{N_service}")  


 
Capella_const
Confidence Interval TAT(hours): (11.005598412092185, 12.39942897149645)
Prediction Interval TAT(hours): (6.725543101043617, 16.67948428254502)
Confidence Interval Service: (0.8997555756655987, 0.9601507794172269)
Prediction Interval Service: (0.7142991650316548, 1.1456071900511708)
Number of Samples TAT:92.54963158557844
Number of Samples Service:17.37639304836008

 
Grace_fo_1
Confidence Interval TAT(hours): (10.531902542056278, 11.972439377397722)
Prediction Interval TAT(hours): (6.108425605591731, 16.39591631386227)
Confidence Interval Service: (0.9071126773058542, 0.9642214526165109)
Prediction Interval Service: (0.7317479491997879, 1.1395861807225771)
Number of Samples TAT:98.85609839358256
Number of Samples Service:15.536758809734835

 
Grace_fo_2
Confidence Interval TAT(hours): (10.044465806129985, 11.461104011850392)
Prediction Interval TAT(hours): (5.694374731344357, 15.811195086636019)
Confidence Interval Service: (0.9147542161956415, 0.9682871999045988)
Pred

SUVAN CODE \
Note :\
1. Please check the CRN method for checking relative performance(I'm not sure which one to use, currently different random numbers are used for each design alternative) \
2. Also, can you make computing t value using a library rather than keeping it hardcoded?, also change in the above code block for absolute performance \
3. Ouput plots - can you use the same flow as professor's slides for project demonstration?, lmk if you need anything from me

Relative Performance \
Comparing all 3 configuration

Sensitivity Analysis